The following code is an alternative code to the one you can find in https://www.py4e.com/code3/tracks/tracks.py. 
It uploads the file Library.xml in SQLite, parsing every element into a dict.

In [ ]:
import xml.etree.ElementTree as ET
import sqlite3

conn = sqlite3.connect('trackdb.sqlite')
cur = conn.cursor()

cur.executescript('''
DROP TABLE IF EXISTS Artist;
DROP TABLE IF EXISTS Album;
DROP TABLE IF EXISTS Track;

CREATE TABLE Artist (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Album (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id  INTEGER,
    title   TEXT UNIQUE
);

CREATE TABLE Track (
    id  INTEGER NOT NULL PRIMARY KEY 
        AUTOINCREMENT UNIQUE,
    title TEXT  UNIQUE,
    album_id  INTEGER,
    len INTEGER, 
    rating INTEGER, 
    count INTEGER
);
''')

fname = 'Library.xml'
file = ET.parse(fname)
to_parse = file.findall('dict/dict/dict')


lll=['Track ID','Name','Artist', 'Album', 'Play Count', 'Rating', 'Total Time'] 
ddd=dict()

for entry in to_parse:

        for child in entry:
            
            if child.tag == 'key' and child.text in lll:

                k = child.text

            elif k in lll:
                
                ddd[k] = child.text
                
                k=None
                
            else: continue


        cur.execute('''INSERT OR IGNORE INTO Artist (name) 
            VALUES ( ? )''', (ddd['Artist'],))

        cur.execute('SELECT id FROM Artist WHERE name = ? ', (ddd['Artist'],))
        artist_id = cur.fetchone()[0] #foreign key
        print(ddd['Artist'])
        print(artist_id)
        cur.execute('''INSERT OR IGNORE INTO Album (title, artist_id)
            VALUES ( ?, ? )''', (ddd['Album'], artist_id))
        cur.execute('SELECT id FROM Album WHERE title = ? ', (ddd['Album'],))
        album_id = cur.fetchone()[0] #foreign key

        cur.execute('''INSERT OR REPLACE INTO Track
            (title, album_id, len, rating, count)
            VALUES ( ?, ?, ?, ?, ? )''',
                    (ddd['Name'], album_id, ddd['Total Time'], ddd['Rating'], ddd['Play Count']))

        conn.commit()